## Process Orders Data 
1. Ingest the data into the data lakehouse - bronze_orders
1. Perform data quality checks and transform the data as required - silver_orders_clean
1. Explode the items array from the order object - silver_orders

In [0]:
%sql

-- 1.

CREATE OR REFRESH STREAMING TABLE bronze_orders
COMMENT "Raw orders ingested from source system"
TBLPROPERTIES ("quality" = "bronze")
AS
SELECT *,
_metadata.file_path AS file_path,
current_timestamp AS ingestion_timestamp
FROM cloud_files(
  "/Volumes/circuitbox/landing/operational_data/orders/",
  "json",
  map("cloudFiles.inferColumnTypes", "true")
);

In [0]:
-- 2.

CREATE OR REFRESH STREAMING TABLE silver_orders_clean(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT vaild_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT vaild_order_status EXPECT (order_status IN ('Pending', 'Shipped', 'Cancelled', 'Completed')),
  CONSTRAINT valid_payment_method EXPECT (payment_method IN ('Credit Card', 'PayPal', 'Bank Transfer')) 
)
COMMENT "Cleaned orders data"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT order_id,
       customer_id,
       CAST(order_timestamp AS TIMESTAMP) AS order_timestamp,
       payment_method,
       items,
       order_status
  FROM STREAM(LIVE.bronze_orders);

In [0]:
-- 3.

CREATE STREAMING TABLE silver_orders
AS
SELECT order_id,
       customer_id,
       order_timestamp,
       payment_method,
       order_status,
       item.item_id,
       item.name AS item_name,
       item.price AS item_price,
      item.quantity AS item_quantity,
      item.category AS item_category
  FROM (SELECT order_id,
              customer_id,
              order_timestamp,
              payment_method,
              order_status,
              explode(items) AS item
          FROM STREAM(LIVE.silver_orders_clean));